<a href="https://colab.research.google.com/github/jmsmg/TIL/blob/main/Python/Concurrency.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 병행성(Concurrency)

- 병행성, 흐름제어 설명
- 이터레이터(Iterator) : 반복 가능 객체
- 제너레이터(Generator) : 반복 가능한 객체를 생산해내는 역할 
- \_\_iter__, \_\_next__
- 클래스 기반 제너레이터 구현

In [ ]:
# Iterator, Generator
# 파이썬 반복 가능한 객체
# collections, text file, list, dict, set, tuple, unpacking, *args...

# 반복 가능한 이유 -> iter(x) 함수 호출
t = 'ABCD'

print(dir(t))

print('------')

# for c in t:
#   print(c)

print('------')

w = iter(t)

print(next(w))
print(next(w))

while True:
  try:
    print(next(w))
  except StopIteration:
    break

print('------')

# 반복형 확인

from collections import abc

# print(dir(t))
print(hasattr(t, '__iter__'))
print(isinstance(t, abc.Iterable))

['__add__', '__class__', '__contains__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getnewargs__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__mod__', '__mul__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__rmod__', '__rmul__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', 'capitalize', 'casefold', 'center', 'count', 'encode', 'endswith', 'expandtabs', 'find', 'format', 'format_map', 'index', 'isalnum', 'isalpha', 'isascii', 'isdecimal', 'isdigit', 'isidentifier', 'islower', 'isnumeric', 'isprintable', 'isspace', 'istitle', 'isupper', 'join', 'ljust', 'lower', 'lstrip', 'maketrans', 'partition', 'replace', 'rfind', 'rindex', 'rjust', 'rpartition', 'rsplit', 'rstrip', 'split', 'splitlines', 'startswith', 'strip', 'swapcase', 'title', 'translate', 'upper', 'zfill']
------
------
A
B
C
D
------
True
True


In [ ]:
# next
class WordSplitter:
  def __init__(self, text):
    self._idx = 0
    self._text = text.split(' ')

  def __next__(self):
    print('Called __next__')
    try:
      word = self._text[self._idx]
    except IndexError:
      raise StopIteration('Stopped Iteration')
    self._idx += 1
    return word
  
  def __repr__(self):
    return 'WordSplit(%s)' % (self._text)

wi = WordSplitter('Do today what you could do tommorow')
print(next(wi))
print(next(wi))

# Generator 패턴
# 1. 지능형 리스트, 딕셔너리, 집합 -> 데이터 양 증가 후 메모리 사용량 증가 -> 제너레이터 사용 권장
# 2. 단위 실행 가능한 코루틴(coroutine)
# 3. 작은 메모리 조각 사용

class WordSplitGenerator:
  def __init__(self, text):
    self._text = text.split(' ')
  
  def __iter__(self):
    for word in self._text:
      yield word # 제너레이터 # yield로 상태 저장
    return

  def __repr__(self):
    return 'WordSplitGenerator(%s)' % (self._text)

wg = WordSplitGenerator('Do today what you could do tommorow')

wt = iter(wg)

Called __next__
Do
Called __next__
today


- 병행성(Concurrency) : 한 컴퓨터가 여러 일을 동시에 수행 (마지막 지점을 알아야함 = 클로저) -> 단일 프로그램안에서 여러 일을 쉽게 처리

- 병렬성(Parallelism) : 여러 컴퓨터가 여러 작업을 동시에 수행 -> 속도

In [12]:
# Generator Ex1

def generator_ex1():
  print('Start')
  yield 'A Point'
  print('Continue')
  yield 'B Point'
  print('End')

temp = iter(generator_ex1())

# print(temp)
print(next(temp))

for v in generator_ex1():
  pass
  print(v)

# Generator Ex2
temp2 = [x * 3 for x in generator_ex1()]
temp3 = (x * 3 for x in generator_ex1())

print('----')
print(temp2)

for i in temp3:
  print(i)

Start
A Point
Start
A Point
Continue
B Point
End
Start
Continue
End
----
['A PointA PointA Point', 'B PointB PointB Point']
Start
A PointA PointA Point
Continue
B PointB PointB Point
End


In [ ]:
# Generator Ex3(중요 함수)
# filterfalse, accumulate,